# QM9 KRR Benchmark

Code to perform KRR on the QM9 dataset using different Nystrom methods, using 100k randomly selected molecules as training points. The l1 Laplace kernel is used with a bandwidth 5120, and the regularization parameter is 1e-8; both were chosen using cross-validation. This code was used to produce Figure 3 in the manuscript together with `matlab_plotting/make_krr_plots.m`

In [1]:
# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import qml, os
from scipy.io import savemat, loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.express as px
import plotly.colors as colors

from KRR_Nystrom import KRR_Nystrom
import rpcholesky
import leverage_score
import unif_sample
import matplotlib.pyplot as plt
import time
from functools import partial
import pickle
# util for parallelizing run trials
from joblib import Parallel, delayed

# kernel thinning
# utils for kernel ridge regression
from goodpoints.krr.util_estimators import get_estimator

`eigenpro2` is not installed...
Using `torch.linalg.solve` for training the kernel model

          and may cause an `Out-of-Memory` error
`eigenpro2` is a more scalable solver. To use, pass `method="eigenpro"` to `model.fit()`
To install `eigenpro2` visit https://github.com/EigenPro/EigenPro-pytorch/tree/pytorch/


/home/ag2435/.conda/envs/rpcholesky/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import joblib
print(joblib.__version__)
print(joblib.cpu_count())

1.2.0
8


In [4]:
# add this to be able to render plotly plots in non-vscode notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [5]:

def get_molecules(directory = "molecules/", max_atoms = 29, max_mols = np.Inf, output_index = 7):
    compounds = []
    energies = []
    for f in sorted(os.listdir("molecules/")):
        if len(compounds) >= max_mols:
            break

        try:
            mol = qml.Compound(xyz="molecules/"+f)
            mol.generate_coulomb_matrix(size=max_atoms, sorting="row-norm")
            with open("molecules/"+f) as myfile:
                line = list(myfile.readlines())[1]
                energies.append(float(line.split()[output_index]) * 27.2114) # Hartrees to eV
            compounds.append(mol)
        except ValueError:
            pass
    
    c = list(zip(compounds, energies))
    np.random.shuffle(c)
    compounds, energies = zip(*c)

    X = np.array([mol.representation for mol in compounds])
    Y = np.array(energies).reshape((X.shape[0],1))

    return X, Y 
    

In [6]:
if not os.path.isfile("data/homo.mat"):
    X, Y = get_molecules()
    data = { "X" : X, "Y" : Y }
    savemat("data/homo.mat", data)
else:
    data = loadmat("data/homo.mat")
    
feature = data['X']
target = data['Y'].flatten()
scaler = StandardScaler()
feature = scaler.fit_transform(feature)
n,d = np.shape(feature)


In [7]:
# From KT compress
def log4(n):
    return np.log2(n) / 2
def get_g(n):
    return int( np.ceil( log4(log4(n)) ) ) # Use default value
def largest_power_of_four(n):
    """Returns largest power of four less than or equal to n
    """
    return 4**( (n.bit_length() - 1 )//2)

def get_coreset_size(n, m=1):
    if get_g(n) <= m:
        # with TicToc('compresspp', print_toc=PRINT_TOC):
        # Compress with g'=g+inflation (compressing returns set of size 2^(g+inflation) sqrt(n) )
        # Thin with g'=g (thinning returns set of size 2^inflation sqrt(n) )
        largest_pow_four = largest_power_of_four(n)
        log2n = n.bit_length() - 1
        scale = n // largest_pow_four
        return 2**( 2*(log2n//2) - m ) * scale
    else:
        return int(n / 2**m)

In [8]:

# num_train = 20000
# num_test = 20000 

num_train = 100000
num_test = n - num_train


# ks = range(200, 1200, 200)
sqrt_n = get_coreset_size(num_train, int(log4(num_train)))
ks = [ sqrt_n, sqrt_n* 2, sqrt_n*4 ]
print('ks', ks)

train_sample = feature[:num_train]
train_sample_target = target[:num_train]
test_sample = feature[num_train:num_train+num_test]
test_sample_target = target[num_train:num_train+num_test]


ks [256, 512, 1024]


In [9]:

def mean_squared_error(true, pred):
    return np.mean((true - pred)**2)
def mean_average_error(true, pred):
    return np.mean(np.abs(true - pred))
def SMAPE(true,pred):
    return np.mean(abs(true - pred)/((abs(true)+abs(pred))/2))


In [10]:

methods = { 
    # deterministic methods
    'Greedy' : rpcholesky.greedy, 
    
    # random methods
    'Uniform' : unif_sample.uniform_sample,
    'RPCholesky' : rpcholesky.rpcholesky,
    'RLS' : leverage_score.recursive_rls_acc,
    'block50RPCholesky' : partial(rpcholesky.block_rpcholesky,b=50),

    'kt': None,
    'st' : None
}

num_trials = 100
lamb = 1.0e-8
sigma = 5120.0
result = dict()
# n_jobs = 2 
n_jobs = 8
savepath = f"data/molecule{num_train // 1000}k-trials={num_trials}.pkl"
print(savepath)

data/molecule100k-trials=100.pkl


In [11]:
def train_predict(name, method, train_sample, test_sample, k, idx_k):
    start = time.time()
    if name == 'Greedy':
        model = KRR_Nystrom(kernel = "laplace", 
                    bandwidth = sigma)
        model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
        preds = model.predict_Nystrom(test_sample)
    elif name in ['kt', 'st']: # our methods
        while True:
            try:
                # print(f"Trial {i}")
                model = get_estimator(
                    'regression', 
                    name.lower(), 
                    kernel='laplace',
                    alpha=lamb,
                    sigma=sigma,
                    m=int(log4(num_train))-idx_k,
                )

                model.fit(train_sample, train_sample_target)
                assert len(model.sol_) == k, f"len(model.sol_)={len(model.sol_)} should be same as k={k}"
                preds = model.predict(test_sample)

                break
            except np.linalg.LinAlgError:
                continue
    else:
        while True:
            try:
                # print(f"Trial {i}")
                # # Original
                # model = KRR_Nystrom(kernel = "gaussian", bandwidth = sigma)
                # # Bug fix
                model = KRR_Nystrom(kernel = "laplace", bandwidth = sigma)
                
                model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
                preds = model.predict_Nystrom(test_sample)
                break
            except np.linalg.LinAlgError:
                continue
    end = time.time()
    return preds, end - start
    # return preds

In [12]:

solve_method = 'Direct'

for name, method in methods.items():
    result[name] = dict()
    print(f'------------- Method: {name} -------------')
    result[name]["trace_errors"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMSE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMAE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRSMAPE"] = {}   #np.zeros((len(ks),2))
    result[name]["queries"] = {}    #np.zeros((len(ks),2))

    for idx_k in range(len(ks)):
        k = ks[idx_k]
        print(f'k = {k}')
        trace_err = []
        runtime = []
        queries = []
        KRRmse = []
        KRRmae = []
        KRRsmape = []

        if name == 'Greedy':
            trials = 1 # deterministic
        else:
            trials = num_trials # stochastic

        parallel = Parallel(n_jobs= n_jobs) #, return_as="generator") need joblib>=1.3 for return_as functionality
        output_generator = parallel(delayed(train_predict)(
            name, 
            method, 
            train_sample, 
            test_sample,
            k,
            idx_k,
        ) for _ in range(trials))

        for preds, elapsed_time in output_generator:
            KRRmse.append(mean_squared_error(test_sample_target, preds))
            KRRmae.append(mean_average_error(test_sample_target, preds))
            KRRsmape.append(SMAPE(test_sample_target, preds))
            # queries.append(model.queries)
            # trace_err.append(model.reltrace_err) 
            
            # TODO: placeholder for now
            queries.append(np.nan)
            trace_err.append(np.nan)

            print(f'KRR acc: mse {KRRmse[-1]}, mae {KRRmae[-1]}, smape {KRRsmape[-1]}')
            # print(f'time: sample {model.sample_time}, linsolve {model.linsolve_time}, pred {model.pred_time}')
            print(f'time: {elapsed_time}')
            
        result[name]["trace_errors"][k] = trace_err   # [np.mean(trace_err),np.std(trace_err)]
        result[name]["KRRMSE"][k] = KRRmse    # [np.mean(KRRmse),np.std(KRRmse)]
        result[name]["KRRMAE"][k] = KRRmae    #[np.mean(KRRmae),np.std(KRRmae)]
        result[name]["KRRSMAPE"][k] = KRRsmape    #[np.mean(KRRsmape),np.std(KRRsmape)]
        result[name]["queries"][k] = queries  #[np.mean(queries)/float(num_train**2),np.std(queries)/float(num_train**2)]

        # savemat("data/{}_molecule100k.mat".format(name), result[name])
        # use pickle to save the result periodically
        with open(savepath, 'wb') as f:
            pickle.dump(result, f)


------------- Method: Greedy -------------
k = 256


KRR acc: mse 0.2953552168387065, mae 0.4035185379305591, smape 0.06261773763928957
time: 39.453399658203125
k = 512


KRR acc: mse 0.24421373487415501, mae 0.36844320535362246, smape 0.05722567879137312
time: 81.02892112731934
k = 1024


KRR acc: mse 0.19397560568887487, mae 0.329450039528845, smape 0.05118195187590928
time: 173.9728467464447
------------- Method: Uniform -------------
k = 256


KRR acc: mse 0.22343088147622003, mae 0.350080467365929, smape 0.05442110121672776
time: 10.7155921459198
KRR acc: mse 0.2230129292819063, mae 0.3494556090531451, smape 0.05429602304705962
time: 10.601705074310303
KRR acc: mse 0.22540637721269666, mae 0.3521572927883694, smape 0.05473928119766837
time: 10.754739999771118
KRR acc: mse 0.22136375513269888, mae 0.34923754360501186, smape 0.05425512593196117
time: 10.733570337295532
KRR acc: mse 0.21442888385545614, mae 0.3432807032223982, smape 0.053369637680714364
time: 10.873241186141968
KRR acc: mse 0.2157875924269255, mae 0.3444035218294266, smape 0.05352094965161737
time: 10.817147493362427
KRR acc: mse 0.20868490939111403, mae 0.340180317477612, smape 0.05285615206592838
time: 10.745110750198364
KRR acc: mse 0.21376915877021563, mae 0.34514093847111643, smape 0.0536067613840602
time: 10.839035034179688
KRR acc: mse 0.2081954532904643, mae 0.34003822374478593, smape 0.05284500983836238
time: 10.69798755645752
KRR acc: mse 0.216116320

...

`eigenpro2` is not installed...
Using `torch.linalg.solve` for training the kernel model

          and may cause an `Out-of-Memory` error
`eigenpro2` is a more scalable solver. To use, pass `method="eigenpro"` to `model.fit()`
To install `eigenpro2` visit https://github.com/EigenPro/EigenPro-pytorch/tree/pytorch/


KRR acc: mse 0.2449796885564612, mae 0.3617801040578157, smape 0.056257150213205355
time: 612.8018391132355
KRR acc: mse 0.24026014825971392, mae 0.3603353907105764, smape 0.0560300970416014
time: 625.9755816459656
KRR acc: mse 0.24521589820027467, mae 0.36548426159429537, smape 0.05682489377650409
time: 540.1509802341461
KRR acc: mse 0.2386337863855084, mae 0.36116438493964526, smape 0.056172354771911834
time: 633.4110040664673
KRR acc: mse 0.2351859574404182, mae 0.359838633021613, smape 0.05595506769003765
time: 605.8466169834137
KRR acc: mse 0.24689038130666238, mae 0.3695654437097423, smape 0.05745247380886873
time: 638.1866703033447
KRR acc: mse 0.24426902152993338, mae 0.36623310995865227, smape 0.056937634595536454
time: 637.9279382228851
KRR acc: mse 0.24190903763590738, mae 0.36361813762684486, smape 0.05656339873237848
time: 580.8703162670135
KRR acc: mse 0.23623999638891935, mae 0.3594968492246981, smape 0.05591797667236665
time: 525.363608121872
KRR acc: mse 0.249523628138

KRR acc: mse 0.30352164930348413, mae 0.40585583854316853, smape 0.0630255233254316
time: 0.30269742012023926
KRR acc: mse 0.28902236200212605, mae 0.3955031774611388, smape 0.06142254157100841
time: 0.3037123680114746
KRR acc: mse 0.3229577597908488, mae 0.42151224494418515, smape 0.06539375047288618
time: 0.30364227294921875
KRR acc: mse 0.31954895375791337, mae 0.4167844505079748, smape 0.06466513899991212
time: 0.3022754192352295
KRR acc: mse 0.2977470504593079, mae 0.40414681994955404, smape 0.06274862628185432
time: 0.29883766174316406
KRR acc: mse 0.3073525070027638, mae 0.4109355422444997, smape 0.06381606606728867
time: 0.3041951656341553
KRR acc: mse 0.31175023905110005, mae 0.40799204649041176, smape 0.06329319884041494
time: 0.3370335102081299
KRR acc: mse 0.32053351684629894, mae 0.4272132091645062, smape 0.0663115464223345
time: 0.33778929710388184
KRR acc: mse 0.3079533940324442, mae 0.4095152215673294, smape 0.06356080273486374
time: 0.30503201484680176
KRR acc: mse 0.3

KRR acc: mse 0.27771200376080707, mae 0.3892323120573496, smape 0.060444604998348854
time: 0.5588119029998779
KRR acc: mse 0.2737291112776941, mae 0.38774724152601286, smape 0.0602787324185591
time: 0.5609698295593262
KRR acc: mse 0.2739404012217588, mae 0.3849252291122066, smape 0.05984080906719013
time: 0.5598971843719482
KRR acc: mse 0.27279373350940145, mae 0.384948113023668, smape 0.05980506551035177
time: 0.5611460208892822
KRR acc: mse 0.27954243917098914, mae 0.39577243907050924, smape 0.06151396253876678
time: 0.5633182525634766
KRR acc: mse 0.2785614912406888, mae 0.3952108953497294, smape 0.06136333749813395
time: 0.5636563301086426
KRR acc: mse 0.28215695524088913, mae 0.3922358094090068, smape 0.06095865190162618
time: 0.5533661842346191
KRR acc: mse 0.28929327032042235, mae 0.39794823594654216, smape 0.06183504976653982
time: 0.5563220977783203
KRR acc: mse 0.28745646914953965, mae 0.40168843502742385, smape 0.0623824531339162
time: 0.5580735206604004
KRR acc: mse 0.28186

KRR acc: mse 0.23741303038469186, mae 0.3611944614381893, smape 0.056186569310182684
time: 1.0845158100128174
KRR acc: mse 0.24129646293527932, mae 0.3666698159902612, smape 0.05699510617712132
time: 1.0794346332550049
KRR acc: mse 0.2404881100487247, mae 0.3648230244371633, smape 0.05671080408582734
time: 1.077453374862671
KRR acc: mse 0.24408409853356805, mae 0.362526046127279, smape 0.05637625641218269
time: 1.1043992042541504
KRR acc: mse 0.2585642972136433, mae 0.3759216350766435, smape 0.05845126013499966
time: 1.0705580711364746
KRR acc: mse 0.24902814858747513, mae 0.3688377657859851, smape 0.05733951655883235
time: 1.0691072940826416
KRR acc: mse 0.23936462982366583, mae 0.3607624688054441, smape 0.056081668506378725
time: 1.0879130363464355
KRR acc: mse 0.24665438178655116, mae 0.36862501719926627, smape 0.05732299940278137
time: 1.0959758758544922
KRR acc: mse 0.24525463266585099, mae 0.3644088070870553, smape 0.0567158777888392
time: 1.079817295074463
KRR acc: mse 0.2461094

In [13]:
result

{'Greedy': {'trace_errors': {256: [nan], 512: [nan], 1024: [nan]},
  'KRRMSE': {256: [0.2953552168387065],
   512: [0.24421373487415501],
   1024: [0.19397560568887487]},
  'KRRMAE': {256: [0.4035185379305591],
   512: [0.36844320535362246],
   1024: [0.329450039528845]},
  'KRRSMAPE': {256: [0.06261773763928957],
   512: [0.05722567879137312],
   1024: [0.05118195187590928]},
  'queries': {256: [nan], 512: [nan], 1024: [nan]}},
 'Uniform': {'trace_errors': {256: [nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    

## Plot results

In [14]:
with open(savepath, "rb") as f:
    result = pickle.load(f)
# with open("data/molecule20k-kt.pkl", "rb") as f:
#     result2 = pickle.load(f)
# concatenate the results
# result.update(result2)

In [15]:
# plot line graph with error bars
metrics = ['KRRSMAPE', 'KRRMSE']
fig = make_subplots(rows=len(metrics), cols=1, subplot_titles=metrics, shared_xaxes=True, vertical_spacing=0.1)
colors_list = colors.qualitative.Plotly 
# * (
#     len(model_names) // len(colors.qualitative.Plotly) + 1
# )
model_names = list(result.keys())

for name in result.keys():
    color = colors_list[model_names.index(name)]
    for r, metric in enumerate(metrics):
        if name in ['kt', 'st']:
            for k, vals in result[name][metric].items():
                # print(k, krrmse)
                fig.add_trace(go.Box(
                    x=[k] *len(vals),
                    y=vals,
                    name=name,
                    # opacity=0.5,
                    legendgroup=name,
                    # line_color=color,
                    # offsetgroup=model_name_prefix,
                    # showlegend=color not in colors_used,
                    boxmean=True,
                    line_color = color
                ), row=r+1, col=1)
        else:
            means = [np.mean(krrmse) for _, krrmse in result[name][metric].items() ]
            stds = [np.std(krrmse) for _, krrmse in result[name][metric].items() ]
            # print(ks, means, stds)
            fig.add_trace(go.Scatter(
                x=ks, 
                y=means, 
                mode='lines+markers', 
                name=name, 
                error_y=dict(
                    type='data', # value of error bar given in data coordinates
                    array=stds,
                    visible=True
                ),
                legendgroup=name,
                line_color = color
            ), row=r+1, col=1)
    fig.update_xaxes(title_text="k", row=r+1, col=1)
        
fig.update_layout(title='MSE vs. k (columns)',
                    #  xaxis_title='Coreset Size',
                    #  yaxis_title='MSE',
                     height=800, width=800)
fig.show()